In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2

In [3]:
%load_ext sql

In [4]:
locator = Nominatim(user_agent="myGeocoder") # nie wypierdalać 

In [5]:
def get_data(link):
    try:
        html_data = requests.get(link)
        html_data.content
        soup = BeautifulSoup(html_data.content, 'html5lib')
    except Exception:
        return None 

    try:
        price = soup.find('strong',attrs={"data-cy":"adPageHeaderPrice"}).text
        price = price.replace("zł","").replace(" ","")
        price = float(price)

    except Exception:
        price = np.NaN

    try:
        m2_price = soup.find('div',attrs={"aria-label":"Cena za metr kwadratowy"}).text
        m2_price = m2_price.replace("zł/m²","").replace(" ","")
        m2_price = float(m2_price)
    except Exception:
        m2_price = np.NaN

    try:
        rent = soup.find('div',attrs={"aria-label":"Czynsz"}).find_all('div')[1].text
        rent = rent.replace("zł","").replace(" ","")
        rent = float(rent)
    except Exception:
        rent = np.NaN

    try:
        m2 = soup.find('div',attrs={"aria-label":"Powierzchnia"}).find_all('div')[1].text
        m2 = m2.replace("m²","").replace(",",".")
        m2 = float(m2)
    except Exception:
        m2 = np.NaN

    try:
        Type_of_building = soup.find('div',attrs={"aria-label":"Rodzaj zabudowy"}).find_all('div')[1].text
        str(Type_of_building)
    except Exception:
        Type_of_building = np.NaN

    try:
        room_number = soup.find('div',attrs={"aria-label":"Liczba pokoi"}).find_all('div')[1].text
        room_number = int(room_number)
    except Exception:
        room_number = np.NaN

    try:
        Building_material = soup.find('div',attrs={"aria-label":"Materiał budynku"}).find_all('div')[1].text
        Building_material = str(Building_material)
    except Exception:
        Building_material = np.NaN
        
    try:
        Year_of_construction = soup.find('div',attrs={"aria-label":"Rok budowy"}).find_all('div')[1].text
        Year_of_construction = int(Year_of_construction)
    except Exception:
        Year_of_construction = np.NaN
    
    try:
        floor = soup.find('div',attrs={"aria-label":"Piętro"}).find_all('div')[1].text
        floor = int(floor)
    except Exception:
        floor = np.NaN
        
    try:
        windows = soup.find('div',attrs={"aria-label":"Okna"}).find_all('div')[1].text
        windows = str(windows)
    except Exception:
        windows = np.NaN
    
    try:
        Form_of_the_property = soup.find('div',attrs={"aria-label":"Forma własności"}).find_all('div')[1].text
        Form_of_the_property = str(Form_of_the_property)
    except Exception:
        Form_of_the_property = np.NaN
    
    try:
        Market = soup.find('div',attrs={"aria-label":"Rynek"}).find_all('div')[1].text
        Market = str(Market)
    except Exception:
        Market = np.NaN
    
    try:
        Number_of_floors = soup.find('div',attrs={"aria-label":"Liczba pięter"}).find_all('div')[1].text
        Number_of_floors = int(Number_of_floors)
    except Exception:
        Number_of_floors = np.NaN
        
    try:
        Heating = soup.find('div',attrs={"aria-label":"Ogrzewanie"}).find_all('div')[1].text
        Heating = str(Heating)
    except Exception:
        Heating = np.NaN
        
    media = np.NaN
    tab_all = soup.find('div',attrs={"class":"css-1lw3ul3 ex3yvbv4"})
    try:
        for two in tab_all.find_all('div'):
            tab_all_deep = two.find_all('h3')[0].text
            if tab_all_deep == "media":
                tab_all_deep_2 = two.find_all('ul')[0]
                media = ""
                for one in tab_all_deep_2.find_all('li'):
                    media += one.text + ", "
                media = media[:-2]
    except:
        pass


        
    security = np.NaN
    tab_all = soup.find('div',attrs={"class":"css-1lw3ul3 ex3yvbv4"})
    try:
        for two in tab_all.find_all('div'):
            tab_all_deep = two.find_all('h3')[0].text
            if tab_all_deep == "zabezpieczenia":
                tab_all_deep_2 = two.find_all('ul')[0]
                security = ""
                for one in tab_all_deep_2.find_all('li'):
                    security += one.text + ", "
                security = security[:-2]
    except:
        pass

        
    Additional_information = np.NaN
    tab_all = soup.find('div',attrs={"class":"css-1lw3ul3 ex3yvbv4"})
    try:
        for two in tab_all.find_all('div'):
            tab_all_deep = two.find_all('h3')[0].text
            if tab_all_deep == "informacje dodatkowe":
                tab_all_deep_2 = two.find_all('ul')[0]
                Additional_information = ""
                for one in tab_all_deep_2.find_all('li'):
                    Additional_information += one.text + ", "
                Additional_information = Additional_information[:-2]
    except:
        pass

         
    Equipment = np.NaN
    tab_all = soup.find('div',attrs={"class":"css-1lw3ul3 ex3yvbv4"})
    try:
        for two in tab_all.find_all('div'):
            tab_all_deep = two.find_all('h3')[0].text
            if tab_all_deep == "wyposażenie":
                tab_all_deep_2 = two.find_all('ul')[0]
                Equipment = ""
                for one in tab_all_deep_2.find_all('li'):
                    Equipment += one.text + ", "
                Equipment = Equipment[:-2]
    except:
        pass

    try:
        link_2 = link
        link_2 = str(link_2)
    except Exception:
        pass
    
    try:
        location = soup.find('a',attrs={"class":"css-1ibwe9h e1nbpvi62"}).text
    except Exception:
        location = np.NaN
    
    try:
        location = locator.geocode(location)
        geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
        latitude = location.latitude
        longitude = location.longitude
    except Exception:
        latitude = np.NaN
        longitude = np.NaN
    

        
    return {'price': price, 'm2_price': m2_price, 'rent': rent, 'm2': m2, 'Type_of_building': Type_of_building,
           'room_number':room_number,'Building_material': Building_material, 'Year_of_construction': Year_of_construction,
            'floor': floor, 'windows': windows, 'Form_of_the_property': Form_of_the_property, 'Market' : Market,
           'Number_of_floors': Number_of_floors, 'Heating': Heating,
            'media': media,'link_2': link_2, 'security': security, 'Additional_information': Additional_information,
            'Equipment': Equipment, 'location': location, 'latitude': latitude, 'longitude': longitude}

In [6]:
table = {'price': [], 'm2_price': [], 'rent': [], 'm2': [], 'Type_of_building': [],
        'room_number': [],'Building_material': [], 'Year_of_construction': [],
        'floor': [], 'windows': [], 'Form_of_the_property': [],'Market': [],
         'Number_of_floors': [], 'Heating': [], 'media': [], 'security': [], 'link_2': [],
         'Additional_information': [], 'Equipment': [], 'location': [], 'latitude': [], 'longitude': []}

In [7]:
df_table = pd.DataFrame(table)

print(df_table)

Empty DataFrame
Columns: [price, m2_price, rent, m2, Type_of_building, room_number, Building_material, Year_of_construction, floor, windows, Form_of_the_property, Market, Number_of_floors, Heating, media, security, link_2, Additional_information, Equipment, location, latitude, longitude]
Index: []

[0 rows x 22 columns]


In [8]:
Link_ERROR = 0
Read_announcements = 0
#page = 1
#while True:

for page in range(1,10):  
    try:
        html_data_link = requests.get(f"https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page={page}")
        page += 1
    except Exception:
        
        print("Link_ERROR")
        continue
    
    soup_link = BeautifulSoup(html_data_link.content, 'html5lib')
    
    if soup_link.find('div',attrs={"class":"css-pi2gf8 e1qwpsp45"}) is None:
        pass
    else:
        break

    for offer in soup_link.find_all('a',attrs={"data-cy":"listing-item-link"}):
            new_row = get_data("https://www.otodom.pl"+ offer['href'])
            if new_row == None:
                Link_ERROR += 1
            else:
                df_table = df_table.append(new_row, ignore_index=True)
                Read_announcements += 1
                clear_output(wait=True)
                print("Read_announcements ", Read_announcements)
                print("Link_ERROR ",Link_ERROR)
            
print("Finish")

Read_announcements  243
Link_ERROR  0
Finish


In [9]:
df_table.head(10)

,price,m2_price,rent,m2,Type_of_building,room_number,Building_material,Year_of_construction,floor,windows,...,Number_of_floors,Heating,media,security,link_2,Additional_information,Equipment,location,latitude,longitude
0,583000.0,14987.0,NaN,38.90,blok,2.0,beton,1970.0,4.0,drewniane,...,15.0,miejskie,NaN,domofon / wideofon,https://www.otodom.pl/pl/oferta/38m-w-scislym-...,"piwnica, winda","zmywarka, lodówka, meble, piekarnik, kuchenka,...","(Śródmieście, Warszawa, województwo mazowiecki...",52.232810,21.019067
1,985000.0,10051.0,NaN,98.00,szeregowiec,4.0,cegła,2013.0,NaN,plastikowe,...,2.0,gazowe,NaN,NaN,https://www.otodom.pl/pl/oferta/sprzedam-miesz...,"balkon, garaż/miejsce parkingowe, ogródek, tar...",NaN,"(Pod Brzozami, Zbytki, Zerzeń, Wawer, Warszawa...",52.198891,21.120890
2,770000.0,13464.0,780.0,57.19,blok,2.0,żelbet,2000.0,3.0,plastikowe,...,11.0,miejskie,"telewizja kablowa, internet, telefon","drzwi / okna antywłamaniowe, teren zamknięty, ...",https://www.otodom.pl/pl/oferta/przestronne-ci...,"balkon, garaż/miejsce parkingowe, winda","lodówka, meble, kuchenka, pralka","(Esej, Chomiczówka, Bielany, Warszawa, wojewód...",52.278832,20.914818
3,729000.0,17357.0,500.0,42.00,kamienica,2.0,cegła,1971.0,6.0,plastikowe,...,6.0,miejskie,"telewizja kablowa, internet, telefon","drzwi / okna antywłamaniowe, domofon / wideofo...",https://www.otodom.pl/pl/oferta/al-jerozolimsk...,"klimatyzacja, winda","zmywarka, lodówka, meble, kuchenka, telewizor,...","(Marszałkowska, Centrum, Śródmieście Północne,...",52.235026,21.008465
4,1165000.0,16958.0,800.0,68.70,apartamentowiec,3.0,inne,2017.0,NaN,plastikowe,...,7.0,miejskie,NaN,"system alarmowy, domofon / wideofon, monitorin...",https://www.otodom.pl/pl/oferta/sprzedaz-miesz...,"garaż/miejsce parkingowe, ogródek",NaN,"(Mokotów, Warszawa, województwo mazowieckie, P...",52.193987,21.045781
5,459000.0,11769.0,NaN,39.00,NaN,2.0,NaN,1955.0,4.0,NaN,...,4.0,NaN,NaN,NaN,https://www.otodom.pl/pl/oferta/kamienica-plac...,piwnica,NaN,"(Praga-Północ, Warszawa, województwo mazowieck...",52.264884,21.027344
6,910000.0,15965.0,650.0,57.00,apartamentowiec,2.0,cegła,2011.0,7.0,drewniane,...,11.0,miejskie,"telewizja kablowa, internet, telefon","system alarmowy, drzwi / okna antywłamaniowe, ...",https://www.otodom.pl/pl/oferta/2-p-mokotow-ga...,"balkon, garaż/miejsce parkingowe, winda","zmywarka, lodówka, meble, piekarnik, kuchenka,...","(Mokotów, Warszawa, województwo mazowieckie, P...",52.193987,21.045781
7,1433850.0,16500.0,860.0,86.90,kamienica,3.0,cegła,1936.0,2.0,plastikowe,...,4.0,miejskie,"telewizja kablowa, internet, telefon","system alarmowy, drzwi / okna antywłamaniowe, ...",https://www.otodom.pl/pl/oferta/3-p-mokotow-ka...,"balkon, piwnica, winda",NaN,"(Mokotów, Warszawa, województwo mazowieckie, P...",52.193987,21.045781
8,585000.0,11700.0,NaN,50.00,blok,2.0,wielka płyta,1972.0,5.0,NaN,...,10.0,NaN,NaN,NaN,https://www.otodom.pl/pl/oferta/przestronne-ci...,"balkon, piwnica, garaż/miejsce parkingowe, win...",meble,"(Mieczysława Wolfkego, Bemowo-Lotnisko, Bemowo...",52.256629,20.926194
9,1187058.0,18490.0,NaN,64.20,apartamentowiec,3.0,NaN,2009.0,NaN,NaN,...,22.0,miejskie,NaN,monitoring / ochrona,https://www.otodom.pl/pl/oferta/xvi-pietro-64m...,"garaż/miejsce parkingowe, winda",NaN,"(Grzybowska, Za Żelazną Bramą, Śródmieście Pół...",52.237039,21.000487


In [10]:
table_name = 'Warsaw_sell_full_2'

In [11]:
df_table.to_csv (table_name +'.csv', index = False, header=True)

In [12]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otodom', user='barto', password='biznes')

In [13]:
cur = psql.cursor()

In [14]:
cur.execute('DROP TABLE IF EXISTS '+ table_name)
createTableCommand = 'CREATE TABLE ' + table_name + '''
    (ID int NOT NULL,
    price float,
    m2_price float,
    rent float,
    m2 float,
    Type_of_building VARCHAR,
    room_number float,
    Building_material VARCHAR,
    Year_of_construction float,
    floor float,
    windows VARCHAR,
    Form_of_the_property VARCHAR,
    Market VARCHAR,
    Number_of_floors float,
    Heating VARCHAR,
    media VARCHAR,
    security VARCHAR,
    link_2 VARCHAR,
    Additional_information VARCHAR,
    Equipment VARCHAR,
    location VARCHAR,
    latitude float,
    longitude float,
    PRIMARY KEY (ID)
    )'''
cur.execute(createTableCommand)
psql.commit()

In [15]:
insert_table = ['ID',
    'price',
    'm2_price',
    'rent',
    'm2',
    'Type_of_building',
    'room_number',
    'Building_material',
    'Year_of_construction',
    'floor',
    'windows',
    'Form_of_the_property',
    'Market',
    'Number_of_floors',
    'Heating',
    'media',
    'security',
    'link_2',
    'Additional_information',
    'Equipment',
    'location',
    'latitude',
    'longitude']


a = 0
for row in df_table.itertuples():
    full=""
    full_fit="INSERT INTO " + table_name + "("
    for (column, insert) in zip(row, insert_table):
        if str(column) =='nan':
            continue
        column = str(column)
        column = column.replace("'", "''")
        full +="'"+column+"'"+", "
        full_fit += insert + ', '  
        
    full=full[:-2]
    full_fit=full_fit[:-2]
    full_fit=full_fit+')'
    Value_data='VALUES ('+full+');'
    insertDataCommand = full_fit+"\r\n"+Value_data
    cur.execute(insertDataCommand)
psql.commit()
print("Finish")

Finish
